abrimos el json


In [1]:
# Con estas 2 lineas recargamos automaticamente todos los archivos afiliados al proyecto, sin necesidad de usar el boton de Restart
%load_ext autoreload
%autoreload 2

from modules.open_json import open_json
import folium
 
jsons = list(open_json("../json/mypimes.json"))
geojson = open_json("../json/lha.geojson")

provincia_map = folium.Map(location=(23.133865,-82.3585654))

for i in range(len(jsons)):
        
    folium.Marker(
        location=[jsons[i]["ubicacion"]["lat"], jsons[i]["ubicacion"]["long"]],
        tooltip=f"{jsons[i]["nombre"]}",
        popup= f"<h1>Municipio: {jsons[i]["municipio"]}✅  Horario: {jsons[i]["horario"]} ⏰</h1>",
        icon=folium.Icon(icon="shopping-cart", color="blue")
    ).add_to(provincia_map)
    
folium.GeoJson(geojson,
               style_function= lambda x: {
                   "fillColor": "#93FF69",
                   "color": "#08CB00"
               }
               ).add_to(provincia_map)
folium
    
provincia_map

In [ ]:

from modules.graphs import promedio
productos = []
for m_productos in jsons:
    productos = m_productos["productos"]

nacional = []
for marcas_nacional in productos:
    if marcas_nacional["tipo"] == "bebida":
        if marcas_nacional["nacional"] == True:
            nacional.append(marcas_nacional)


        
        



[{'tipo': 'bebida', 'variedad': 'ron', 'nombre': 'habana club', 'marca': 'Ron Havana Club', 'presentacion': 'litros', 'nacional': True}]
